In [2]:
import numpy as np 
np.random.seed(2019)

from numpy import genfromtxt

import random as r
r.seed(2019)

import sys
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,accuracy_score
from sklearn.metrics import confusion_matrix

import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.manual_seed(2019)
torch.cuda.manual_seed_all(2019)

import os
os.environ['PYTHONHASHSEED'] = str(2019)


from sklearn.model_selection import train_test_split
from util import d, here

import pandas as pd
from argparse import ArgumentParser

import random, tqdm, sys, math, gzip
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

torch.backends.cudnn.benchmark = False

torch.backends.cudnn.deterministic = True

import gc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit,StratifiedKFold

from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

import fine_tuned
import inference
import time

import shutil

## Part 1 Built-in Functions

In [11]:
## show size of variables
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

## random over sampling instances belong to minority class in case of imbalanced dataset
def resample(X,Y,random_state=2019):
    X_reshape = np.reshape(X, (X.shape[0], X.shape[1]*X.shape[2]))
    rus = RandomOverSampler(random_state=random_state)
    X_res, y_res = rus.fit_resample(X_reshape, Y)
    X_back = np.reshape(X_res, (X_res.shape[0],X.shape[1],X.shape[2]))
    
    return X_back, y_res

## seed the initial data
def initial_seed_dataset(n_initial, Y,random_state):
    
    np.random.seed(random_state)
    
    df = pd.DataFrame()
    df['label'] = Y

    Samplesize = n_initial  #number of samples that you want       
    initial_samples = df.groupby('label', as_index=False).apply(lambda array: array.loc[np.random.choice(array.index, Samplesize, False),:])

    permutation = [index[1] for index in initial_samples.index.tolist()]
    
    print ('initial random chosen samples', permutation)
    
    return permutation

## compute the entropy
def compute_entropy(output):
    entropy = -(output[:,0]*np.log2(output[:,0]) + output[:,1]*np.log2(output[:,1]))
    return entropy


## sample the candidates for labelling
def sample_candidates(selection_method,permutation,y_prob,num_candidate,y_pred,dist):
    
    print('*'*20,'selection method ',selection_method,'*'*20)
    
        
        
    if selection_method == 'uncertainty':
       
        candidate_index = []
        
        for index in dist.argsort():
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    if selection_method == 'certainty':
        entropy = compute_entropy(np.array(y_prob))
        candidate_index = []
        
        
        for index in entropy.argsort()[:]:
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    if selection_method == 'mostConfident':
        entropy = compute_entropy(np.array(y_prob))
        candidate_index = []
        
        
        for index in np.argsort(np.array(y_prob)[:,1])[::-1]:
            if len(candidate_index) == num_candidate:
                break
            if index not in permutation:
                candidate_index.append(index)

        permutation = permutation+candidate_index
        
    print('*'*20,'num of training set ',len(permutation),'*'*20)   
        
    return permutation

## normalization instance
class Normalize(object):
    
    def normalize(self, X_train, X_val):
        self.scaler = MinMaxScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_val   = self.scaler.transform(X_val)
       
        return (X_train, X_val) 
    
    def inverse(self, X_train, X_val):
        X_train = self.scaler.inverse_transform(X_train)
        X_val   = self.scaler.inverse_transform(X_val)
    
        return (X_train, X_val) 



    
## read data from directory
def read_data(path):

    corpora = []
    for filename in os.listdir(path):

        df_temp = pd.read_csv(path+filename)

        corpora.append(df_temp.text.tolist())

    class_one_len = len(corpora[0])
    class_two_len = len(corpora[1])

    return corpora, class_one_len, class_two_len

## construct training text dataset for fine tuning language model, save in directory data/
def construct_train_set(dataset,permutation):
    print('constructing new text training set.....')
    corpora, class_one_len, class_two_len = read_data('./corpus_data/'+dataset+'/')

    texts = np.array(corpora[0]+corpora[1])
    labels = np.array([0]*class_one_len+[1]*class_two_len)

#     X_train, X_test, y_train, y_test = train_test_split(texts[permutation], labels[permutation], test_size=0.0, random_state=2019)
    X_train, y_train = texts[permutation], labels[permutation]
    
    train_df = pd.DataFrame({'id':range(len(X_train)),'label':y_train,'alpha':['a']*len(X_train),'text':X_train})
#     dev_df = pd.DataFrame({'id':range(len(X_test)),'label':y_test,'alpha':['a']*len(X_test),'text':X_test})
    
    train_df.to_csv('data/train.tsv', sep='\t', index=False, header=False)
#     dev_df.to_csv('data/dev.tsv', sep='\t', index=False, header=False)

## Part 2. Import Inferred embeddings

In [19]:
## choose the bert-like models, options are roberta-base, disilbert-base-uncased, gpt2, xlnet-base-cased, albert-base-v2
text_rep = 'roberta-base'


## input dataset 
dataset = 'ProtonPumpInhibitors'
dir_neg = dataset + '_neg.csv'
dir_pos = dataset + '_pos.csv'

representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
labels = np.array([0]*len(representations_neg)+[1]*len(representations_pos))

## Part 3. Active Learning Process

In [12]:
## number of class for final output
NUM_CLS = 2

def active_process(arg):
    """
    Active learning procedure for Adaptive Tuning active learning model.
    """
    
    ## gridsearch for hyper-parameters tuning for SVM
    if arg.loop%arg.gridsearch_interval==0:
        gridsearch = True
    else:
        gridsearch = False
    
    
    if arg.initial: 
        ## the first round, initialize seed data
        permutation = initial_seed_dataset(arg.n_seedsamples,arg.LABEL_emb,arg.initial_random_seed)
       
        X_train = arg.TEXT_emb[permutation]
        Y_train = arg.LABEL_emb[permutation]

        X_val = arg.TEXT_emb
        Y_val = arg.LABEL_emb
        
        bin_count = np.bincount(Y_train)
        unique = np.unique(Y_train)
        print (
        'initial training set size:',
        Y_train.shape[0],
        'unique(labels):',
        unique,
        'label counts:',
        bin_count
        )

        print('Number of training examples ', len(Y_train))
        
    else:
        
        permutation = arg.permutation
       
        
        X_train = arg.TEXT_emb[permutation]
        Y_train = arg.LABEL_emb[permutation]

        X_val = arg.TEXT_emb
        Y_val = arg.LABEL_emb
        
        bin_count = np.bincount(Y_train)
        unique = np.unique(Y_train)
        print (
        'training set size:',
        Y_train.shape[0],
        'unique(labels):',
        unique,
        'label counts:',
        bin_count
        )

        print('Number of training examples ', len(Y_train))


    evaluation= { 'Confusion matrix':[]}
    
    normalizer = Normalize()
    X_train, X_val = normalizer.normalize(X_train, X_val) 
        
    
    if gridsearch == True:
        ## gridsearch for finding best hyper-parameters
        print('start gridsearch ...')
        parameters = [
                    {'kernel': ['linear'],
                     'C': [ 0.01, 0.1, 1,10]}]

        cv = StratifiedKFold(n_splits=5,random_state=arg.initial_random_seed)
        svc = SVC(probability=True,random_state=2019,class_weight='balanced',max_iter=10000)
        classifier = GridSearchCV(svc, parameters, cv=cv,scoring='accuracy',n_jobs=8,verbose = 0)
        classifier.fit(X_train, Y_train)
        print('best parameters is ', classifier.best_params_)
        best_params_ = classifier.best_params_
       
        
    else:
        ## hyper-parameters from last loop
        best_params_ = arg.best_params_
        kernel = best_params_['kernel']
        C = best_params_['C']
        
        classifier = SVC(probability=True,random_state=2019,class_weight='balanced',C=C,kernel=kernel,max_iter=10000)
        classifier.fit(X_train, Y_train)
        print('best parameters is ', classifier)
        
        
    ## evaluation
        
    y_pred_prob = classifier.predict_proba(X_val)
#     y_pred = np.argmax(y_pred_prob, axis=1)
    y_pred = classifier.predict(X_val)
    y_true = Y_val
    
    numerators = classifier.decision_function(X_val)
    try:
        w_norm = np.linalg.norm(classifier.best_estimator_.coef_)
    except Exception:
        w_norm = np.linalg.norm(classifier.coef_)

    dist = abs(numerators) / w_norm
   

    y_true_remain = np.delete(np.array(y_true),permutation)
    y_pred_remain = np.delete(np.array(y_pred),permutation)
        
    print(classification_report(y_true_remain,y_pred_remain))
   
    tn, fp, fn, tp = confusion_matrix(y_true_remain, y_pred_remain, labels=[0,1]).ravel()
    print('TP_H',bin_count[1],' TN_H',bin_count[0], ' TP_M',tp, ' TN_M',tn, ' FP_M', fp, ' FN_M',fn)
    acc = accuracy_score(y_true_remain, y_pred_remain)
    f_score = f1_score(y_true_remain,y_pred_remain,average='macro')
                
    evaluation['Confusion matrix'].append({'TP_H':bin_count[1],'TN_H':bin_count[0], 'TP_M':tp, 'TN_M':tn, 'FP_M': fp, 'FN_M':fn})

    
    return evaluation, y_pred_prob, y_pred,permutation, best_params_,dist

In [13]:
import easydict
from argparse import ArgumentParser

# Run the main function
# selection_method = 'certainty'
selection_method = 'uncertainty'
# selection_method = 'mostConfident'


if __name__ == "__main__":
    
    df = pd.DataFrame()
    df_permutation = pd.DataFrame()
    
    ## using different random seeds to seed the initial dataset
    random_seeds = [i for i in range(1988,1998)]
    
    start_time = time.time()
    for seed in random_seeds:
        confusion_matrice = []
        entropy = []
        permutations = []
        
        for loop in range(100):
            print('processing ',loop,'th loops---------------')

            ## first 20 loops are identical to normal active learning
            if loop==0:
                
                
                dir_neg = dataset + '_neg.csv'
                dir_pos = dataset + '_pos.csv'
                representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
                representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')
                ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
                labels = np.array([0]*len(representations_neg)+[1]*len(representations_pos))
       
                parser = ArgumentParser()
                args = parser.parse_known_args()[0]
                args = easydict.EasyDict({
                        "n_seedsamples":5,
                        "initial_random_seed":seed,
                        "initial": True,
                        "TEXT_emb": ulti_representations,
                        "LABEL_emb": labels,
                        "gridsearch":True,
                        "loop":loop,
                        "gridsearch_interval":10

                })

                evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


                permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
                permutations.append(permutation)
                
                confusion_matrice.append(evaluation['Confusion matrix'])
                
                
        

            else:
                
                ## start fine tuning
                if loop==20:
                    construct_train_set(dataset,permutation)
                    ## fine tuning
                    checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,'roberta-base',int(len(permutation)/3),1e-5)
                    
                    ## re-infer embeddings
                    inference.inference(dataset,checkpoint,loop)

                    
                    ## save tuned embeddings to local
                    dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
                    dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
                    representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
                    representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

                    ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)

                    for key in accs.keys():
                        if key!= checkpoint:
                            shutil.rmtree(key)

                elif loop%20==0 and loop>20 and loop<80:
                    construct_train_set(dataset,permutation)
                    checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,checkpoint,int(len(permutation)/3),1e-5)
                    inference.inference(dataset,checkpoint,loop)

                    dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
                    dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
                    representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
                    representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

                    ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)

                    for key in accs.keys():
                        if key!= checkpoint:
                            shutil.rmtree(key)

                parser = ArgumentParser()
                args = parser.parse_known_args()[0]
                args = easydict.EasyDict({
                        "n_seedsamples":5,
                        "initial_random_seed":seed,
                        "permutation": permutation,
                        "initial": False,
                        "TEXT_emb": ulti_representations,
                        "LABEL_emb": labels,
                        "best_params_":best_params_,
                        "loop":loop,
                        "gridsearch_interval":10
                })

                evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


                permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
                permutations.append(permutation)
                
                confusion_matrice.append(evaluation['Confusion matrix'])
                
               
                
              
        ## the tp_h, tn_h, tp_m, tn_m, fp_m, fn_m for each loop
        df[seed] = [item[0] for item in confusion_matrice]
        df.to_csv('./results/raw_%s_%s_%s_gridsearch.csv'%(text_rep,dataset,selection_method))


        ## the index of documents for each loop
        df_permutation[seed] = [permutation for permutation in permutations]

        df_permutation.to_csv('./results/permutation_%s_%s_%s_result.csv'%(text_rep,dataset,selection_method))
        
        shutil.rmtree("./outputs/") 
    print("--- %s seconds ---" % (time.time() - start_time))

processing  0 th loops---------------
initial random chosen samples [476, 707, 770, 587, 468, 1026, 1891, 1549, 1101, 1390]
initial training set size: 10 unique(labels): [0 1] label counts: [5 5]
Number of training examples  10
start gridsearch ...
best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.94      0.82      0.88       995
           1       0.84      0.95      0.89       995

    accuracy                           0.88      1990
   macro avg       0.89      0.88      0.88      1990
weighted avg       0.89      0.88      0.88      1990

TP_H 5  TN_H 5  TP_M 943  TN_M 818  FP_M 177  FN_M 52
******************** selection method  uncertainty ********************
******************** num of training set  20 ********************
processing  1 th loops---------------
training set size: 20 unique(labels): [0 1] label counts: [12  8]
Number of training examples  20
best parameters is  SVC(C=0.01, cache_size=2

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       943
           1       0.99      0.99      0.99       957

    accuracy                           0.99      1900
   macro avg       0.99      0.99      0.99      1900
weighted avg       0.99      0.99      0.99      1900

TP_H 43  TN_H 57  TP_M 949  TN_M 929  FP_M 14  FN_M 8
******************** selection method  uncertainty ********************
******************** num of training set  110 ********************
processing  10 th loops---------------
training set size: 110 unique(labels): [0 1] label counts: [65 45]
Number of training examples  110
start gridsearch ...
best parameters is  {'C': 0.1, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       935
           1       0.99      0.99      0.99       955

    accuracy                           0.99      1890
   macro avg       0.99      0.99      0.99      18

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       893
           1       1.00      1.00      1.00       907

    accuracy                           1.00      1800
   macro avg       1.00      1.00      1.00      1800
weighted avg       1.00      1.00      1.00      1800

TP_H 93  TN_H 107  TP_M 904  TN_M 891  FP_M 2  FN_M 3
******************** selection method  uncertainty ********************
******************** num of training set  210 ********************
processing  20 th loops---------------
constructing new text training set.....


I0120 16:36:11.690764 15348 configuration_utils.py:160] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at C:\Users\jh\.cache\torch\transformers\e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.9dad9043216064080cf9dd3711c53c0f11fe2b09313eaa66931057b4bdcaf068
I0120 16:36:11.692757 15348 configuration_utils.py:177] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "binary",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 1,
  "use_bfloat16": false,
  "vocab_size": 50265
}

I0120 16:36:12.444

0.703386

...


training set size: 610 unique(labels): [0 1] label counts: [336 274]
Number of training examples  610
start gridsearch ...
best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       664
           1       1.00      1.00      1.00       726

    accuracy                           1.00      1390
   macro avg       1.00      1.00      1.00      1390
weighted avg       1.00      1.00      1.00      1390

TP_H 274  TN_H 336  TP_M 726  TN_M 662  FP_M 2  FN_M 0
******************** selection method  uncertainty ********************
******************** num of training set  620 ********************
processing  61 th loops---------------
training set size: 620 unique(labels): [0 1] label counts: [338 282]
Number of training examples  620
best parameters is  SVC(C=0.01, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kern

best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       649
           1       1.00      1.00      1.00       641

    accuracy                           1.00      1290
   macro avg       1.00      1.00      1.00      1290
weighted avg       1.00      1.00      1.00      1290

TP_H 359  TN_H 351  TP_M 641  TN_M 647  FP_M 2  FN_M 0
******************** selection method  uncertainty ********************
******************** num of training set  720 ********************
processing  71 th loops---------------
training set size: 720 unique(labels): [0 1] label counts: [356 364]
Number of training examples  720
best parameters is  SVC(C=0.01, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=10000, probability=True, random_state=2019,
    shrinking=True, tol=0.001, verbose=False)
            

best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       644
           1       1.00      1.00      1.00       546

    accuracy                           1.00      1190
   macro avg       1.00      1.00      1.00      1190
weighted avg       1.00      1.00      1.00      1190

TP_H 454  TN_H 356  TP_M 546  TN_M 643  FP_M 1  FN_M 0
******************** selection method  uncertainty ********************
******************** num of training set  820 ********************
processing  81 th loops---------------
training set size: 820 unique(labels): [0 1] label counts: [356 464]
Number of training examples  820
best parameters is  SVC(C=0.01, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=10000, probability=True, random_state=2019,
    shrinking=True, tol=0.001, verbose=False)
            

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       644
           1       1.00      1.00      1.00       456

    accuracy                           1.00      1100
   macro avg       1.00      1.00      1.00      1100
weighted avg       1.00      1.00      1.00      1100

TP_H 544  TN_H 356  TP_M 456  TN_M 643  FP_M 1  FN_M 0
******************** selection method  uncertainty ********************
******************** num of training set  910 ********************
processing  90 th loops---------------
training set size: 910 unique(labels): [0 1] label counts: [356 554]
Number of training examples  910
start gridsearch ...
best parameters is  {'C': 0.01, 'kernel': 'linear'}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       644
           1       1.00      1.00      1.00       446

    accuracy                           1.00      1090
   macro avg       1.00      1.00      1.00    

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       642
           1       1.00      1.00      1.00       368

    accuracy                           1.00      1010
   macro avg       1.00      1.00      1.00      1010
weighted avg       1.00      1.00      1.00      1010

TP_H 632  TN_H 358  TP_M 368  TN_M 642  FP_M 0  FN_M 0
******************** selection method  uncertainty ********************
******************** num of training set  1000 ********************
processing  99 th loops---------------
training set size: 1000 unique(labels): [0 1] label counts: [358 642]
Number of training examples  1000
best parameters is  SVC(C=0.01, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=10000, probability=True, random_state=2019,
    shrinking=True, tol=0.001, verbose=False)
              precision    recall  f1-score   support

      

In [14]:
# import easydict
# from argparse import ArgumentParser

# # Run the main function
# # selection_method = 'certainty'
# selection_method = 'uncertainty'
# # selection_method = 'mostConfident'


# if __name__ == "__main__":
    
#     df = pd.DataFrame()
#     df_permutation = pd.DataFrame()
# #     random_seeds = [1989,1990,1991,1992,1993,1994,1995,1996,1997]
#     random_seeds = [i for i in range(1988,1998)]
    
#     start_time = time.time()
#     for seed in random_seeds:
#         confusion_matrice = []
#         entropy = []
#         permutations = []
        
#         for loop in range(100):
#             print('processing ',loop,'th loops---------------')

#             if loop==0 :
                
#                 dir_neg = dataset + '_neg.csv'
#                 dir_pos = dataset + '_pos.csv'
#                 representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                 representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')
#                 ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
#                 labels = np.array([0]*len(representations_neg)+[1]*len(representations_pos))
       
#                 parser = ArgumentParser()
#                 args = parser.parse_known_args()[0]
#                 args = easydict.EasyDict({
#                         "n_seedsamples":5,
#                         "initial_random_seed":seed,
#                         "initial": True,
#                         "TEXT_emb": ulti_representations,
#                         "LABEL_emb": labels,
#                         "gridsearch":True,
#                         "loop":loop,
#                         "gridsearch_interval":10

#                 })

#                 evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


#                 permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
#                 permutations.append(permutation)
                
#                 confusion_matrice.append(evaluation['Confusion matrix'])
                
                
        

#             else:
                
               
#                 if loop==40:
#                     construct_train_set(dataset,permutation)
#                     checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,'roberta-base',32,1e-6)
#                     inference.inference(dataset,checkpoint,loop)

#                     dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
#                     dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
#                     representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                     representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

#                     ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
#                 elif loop%40==0 and loop>40:
#                     construct_train_set(dataset,permutation)
#                     checkpoint, accs = fine_tuned.fine_tuned(loop,dataset,checkpoint,64,1e-5)
#                     inference.inference(dataset,checkpoint,loop)

#                     dir_neg = dataset + '_tuned_neg_%s.csv'%(loop)
#                     dir_pos = dataset + '_tuned_pos_%s.csv'%(loop)
#                     representations_neg = genfromtxt('./%s_data/'%(text_rep)+dir_neg, delimiter=',')
#                     representations_pos = genfromtxt('./%s_data/'%(text_rep)+dir_pos, delimiter=',')

#                     ulti_representations = np.concatenate((representations_neg,representations_pos),axis=0)
                    
              

#                 parser = ArgumentParser()
#                 args = parser.parse_known_args()[0]
#                 args = easydict.EasyDict({
#                         "n_seedsamples":5,
#                         "initial_random_seed":seed,
#                         "permutation": permutation,
#                         "initial": False,
#                         "TEXT_emb": ulti_representations,
#                         "LABEL_emb": labels,
#                         "best_params_":best_params_,
#                         "loop":loop,
#                         "gridsearch_interval":10
#                 })

#                 evaluation,y_pred_prob,y_pred,permutation,best_params_,dist = active_process(args)


#                 permutation = sample_candidates(num_candidate=10,permutation=permutation,selection_method=selection_method,y_prob=y_pred_prob,y_pred=y_pred,dist=dist)
#                 permutations.append(permutation)
                
#                 confusion_matrice.append(evaluation['Confusion matrix'])
                
#         shutil.rmtree("./outputs/")
                
                
            
#         df[seed] = [item[0] for item in confusion_matrice]
#         df.to_csv('./results/raw_%s_%s_%s_gridsearch.csv'%(text_rep,dataset,selection_method))


#         df_permutation[seed] = [permutation for permutation in permutations]

#         df_permutation.to_csv('./results/permutation_%s_%s_%s_result.csv'%(text_rep,dataset,selection_method))
        
#     print("--- %s seconds ---" % (time.time() - start_time))